In [1]:
using DataFrames
using Plots
using CSV
using Statistics
using DelimitedFiles

szczepienia = DataFrame(CSV.File("vaccination_data.txt"))

mutable struct Person1
    zyje::Bool
    chetny_do_szczepienia::Bool
    zaszczepiony1::Bool
    zaszczepiony2::Bool
    zasz_data_1::Int16 #data jako numer dnia od poczatku trwania pandemii
    zasz_data_2::Int16
    zarazony::Bool
    data_zarazenia::Int16
end



In [2]:
function zaraza(szczepienia, dl_prognozy::Int, chec_szczepienia::Float64)
    """
    funkcja przyjmuje dopasowany zestaw danych, dlugosc prognozy w dniach i chec spoleczenstwa do szczepien. 
    Zwraca tablice o 4 kolumnach z:
    dzienna liczba zarazen, dziennej liczbie smierci, liczbie zaszczepionych 1 i 2 dawka.
    """
    populacja = Int64(maximum(szczepienia[2:end, 22]))
    Polacy = Vector{Person1}(undef, populacja)
    
    for i in 1:populacja
        Polacy[i] = Person1(false, false, false, false, 0, 0 , false, 0)
    end
    
    # Tworzenie przybliżonej obecnej (2021-06-04) sytuacji:
    
    #chec_szczepienia = 0.57 #obecna sytuacja posrod niezaszczepionych osob
    liczba_przypadkow = Int64(szczepienia[457,2])
    liczba_zgonow = Int64(szczepienia[457,4])
    liczba_zaszczepionych1 = Int64(szczepienia[457,14])
    liczba_zaszczepionych2 = Int64(szczepienia[457,15])
    obecni_zarazeni = Int64(sum(szczepienia[450:457,3])) #zalozenie ze obecni zarazeni choruja i roznosza chorobe przez tydzien
    obecni_zarazeni2 = obecni_zarazeni
    
    for i in 1:populacja
        if rand()<(1 - liczba_zgonow/populacja)
            Polacy[i].zyje = true
            if rand()<liczba_zaszczepionych1/populacja
                Polacy[i].chetny_do_szczepienia = true
                Polacy[i].zaszczepiony1 = true
                if rand()<liczba_zaszczepionych2/liczba_zaszczepionych1
                    Polacy[i].zaszczepiony2 = true
                else
                    Polacy[i].zasz_data_1 = rand(422:457) # przyblizony okres pierwszego szczepienia
                end
            else
                if rand()<chec_szczepienia
                    Polacy[i].chetny_do_szczepienia = true
                end
                if obecni_zarazeni2 > 0
                    Polacy[i].zarazony = true
                    Polacy[i].data_zarazenia = rand(450:457)
                    obecni_zarazeni2 -= 1
                end
            end
        end
    end

    # Tworzenie tablicy z danymi na okreslona dlugosc i symulacja zmianiajacej sie sytuacji:
    
    Tablica = zeros(Int64, dl_prognozy+1, 6)
    obecny_dzien = 458
    ozdrowiency = szczepienia[450,3]
    
    #surowosc_obostrzen = szczepienia[457, 21]
    #reprodukcja_wirusa = szczepienia[457, 6]
    
    
                   #dzienne zarazenia         dzienna smierc             liczba_zaszczepionych1  liczba_zaszczepionych2   obecnie zarazeni
    Tablica[1,:] = [Int64(szczepienia[457,3]), Int64(szczepienia[457,5]), liczba_zaszczepionych1, liczba_zaszczepionych2, obecni_zarazeni, ozdrowiency]
    
    #println(obecni_zarazeni) 
    
    for i in 2:dl_prognozy+1
        
        obecni_zarazeni = Tablica[i-1,5]
        szansa_kontaktu_z_zarazonym = 0.15 * obecni_zarazeni/populacja #zalozenie ze spotykamy 10osob dziennie srednio
        skutecznosc1 = 0.5
        skutecznosc2 = 0.9
        dzisiaj_smierc = 0
        dzisiaj_zarazeni = 0
        dzisiaj_zaszczepieni = 0
        dzisiaj_zaszczepieni2 = 0
        ozdrowiency = 0
        
        
        for i in 1:populacja
            if Polacy[i].zyje
                
                if Polacy[i].zarazony                                        #co z zarazonymi/smierc
                    
                    if Polacy[i].data_zarazenia+7 <= obecny_dzien
                        praw = rand()
                        if praw<0.9 # po 7dniach szansa wyzdrowienia 90% kazdego dnia
                            Polacy[i].zarazony = false
                            Polacy[i].data_zarazenia = 0
                            obecni_zarazeni -= 1
                            ozdrowiency += 1
                        elseif praw<0.927 # zakladamy, ze moga umrzec tylko ci co choruja ponad tydzien
                            """
                            3% umiera, wiec po tym tygodniu zostaje to 30%, zatem szukamy x takiego, ze:
                            0.03 = x * szereg((0.1)^i)
                            czyli wychodzi, ze prawdopodobienstwo smierci po 7dniu choroby to 0.27 kazdego dnia
                            """
                            Polacy[i].zyje = false
                            Polacy[i].data_zarazenia = 0
                            dzisiaj_smierc += 1
                            obecni_zarazeni -= 1
                        end
                    end
                        
                    
                
                else                                                         #szansa zachorowania + zaszczepienia(not yet)
                   
                    
                    prawd = rand() # by przyspieszyc program
                    if Polacy[i].zaszczepiony1
                        if Polacy[i].zaszczepiony2                                       #przy 2dawkach
                            if prawd < szansa_kontaktu_z_zarazonym && rand()>skutecznosc2
                                Polacy[i].zarazony = true
                                Polacy[i].data_zarazenia = obecny_dzien
                                obecni_zarazeni += 1
                                dzisiaj_zarazeni += 1
                            end
                        else                                                             #przy 1dawce
                            if prawd < szansa_kontaktu_z_zarazonym && rand()>skutecznosc1
                                Polacy[i].zarazony = true
                                Polacy[i].data_zarazenia = obecny_dzien
                                obecni_zarazeni += 1
                                dzisiaj_zarazeni += 1
                            elseif Polacy[i].zasz_data_1+35 <= obecny_dzien
                                Polacy[i].zaszczepiony2 = true
                                dzisiaj_zaszczepieni += 1
                                dzisiaj_zaszczepieni2 += 1
                            end
                        end
                    else                                                                 #niezaszczepiony
                        if prawd < szansa_kontaktu_z_zarazonym
                            Polacy[i].zarazony = true
                            Polacy[i].data_zarazenia = obecny_dzien
                            obecni_zarazeni += 1
                            dzisiaj_zarazeni += 1
                        elseif Polacy[i].chetny_do_szczepienia == true && Polacy[i].zasz_data_1 != obecny_dzien
                            Polacy[i].zasz_data_1 = obecny_dzien + rand(15:50)                        
                        elseif Polacy[i].zasz_data_1 == obecny_dzien
                            Polacy[i].zaszczepiony1 = true
                            dzisiaj_zaszczepieni += 1
                        end
                    end
                end
            end
        end
        println("test", i) #dla ogladania szybkosci postepu kodu
        #aktualizacja tablicy:
        Tablica[i,:] = [dzisiaj_zarazeni,dzisiaj_smierc,Tablica[i-1,3]+dzisiaj_zaszczepieni,Tablica[i-1,4]+dzisiaj_zaszczepieni2,obecni_zarazeni, ozdrowiency]
        obecny_dzien += 1 
    end
    
    
    return Tablica
end

zaraza (generic function with 1 method)

Ponizej pierwsza symulacja, w ktorej chec zaszczepienie deklaruje az 90% jeszcze nie zaszczepionych.

In [3]:
x1 = zaraza(szczepienia, 365, 0.90)

test2
test3
test4
test5
test6
test7
test8
test9
test10
test11
test12
test13
test14
test15
test16
test17
test18
test19
test20
test21
test22
test23
test24
test25
test26
test27
test28
test29
test30
test31
test32
test33
test34
test35
test36
test37
test38
test39
test40
test41
test42
test43
test44
test45
test46
test47
test48
test49
test50
test51
test52
test53
test54
test55
test56
test57
test58
test59
test60
test61
test62
test63
test64
test65
test66
test67
test68
test69
test70
test71
test72
test73
test74
test75
test76
test77
test78
test79
test80
test81
test82
test83
test84
test85
test86
test87
test88
test89
test90
test91
test92
test93
test94
test95
test96
test97
test98
test99
test100
test101
test102
test103
test104
test105
test106
test107
test108
test109
test110
test111
test112
test113
test114
test115
test116
test117
test118
test119
test120
test121
test122
test123
test124
test125
test126
test127
test128
test129
test130
test131
test132
test133
test134
test135
test136
test137
test138
test139
te

366×6 Matrix{Int64}:
 565  91  14224874   7777048  5642  1202
 624  41  14583016   8135190  4936  1289
 553  17  14761112   8313286  4779   693
 538  24  14939774   8491948  4594   699
 506  28  15117908   8670082  4386   686
 509  25  15296943   8849117  4208   662
 462  11  15476018   9028192  3960   699
 390  21  15655240   9207414  3648   681
 398  15  15834445   9386619  3415   616
 382  16  16013411   9565585  3232   549
 352  14  16191250   9743424  3040   530
 321  16  16369965   9922139  2854   491
 309  22  16549225  10101399  2643   498
   ⋮                                   ⋮
   0   0  20660935  14213109     0     0
   0   0  20660935  14213109     0     0
   0   0  20660935  14213109     0     0
   0   0  20660935  14213109     0     0
   0   0  20660935  14213109     0     0
   0   0  20660935  14213109     0     0
   0   0  20660935  14213109     0     0
   0   0  20660935  14213109     0     0
   0   0  20660935  14213109     0     0
   0   0  20660935  14213109     0  

In [4]:
y1 = DataFrame(
    dzienne_zarazenia = x1[1:end,1],
    dzienna_smierc = x1[1:end,2],
    lacznie_1_dawka = x1[1:end,3],
    lacznie_2_dawka = x1[1:end,4],
    teraz_zarazeni = x1[1:end,5],
    ozdrowiency = x1[1:end,6]
    )

,dzienne_zarazenia,dzienna_smierc,lacznie_1_dawka,lacznie_2_dawka,teraz_zarazeni
,Int64,Int64,Int64,Int64,Int64
1,565,91,14224874,7777048,5642
2,624,41,14583016,8135190,4936
3,553,17,14761112,8313286,4779
4,538,24,14939774,8491948,4594
5,506,28,15117908,8670082,4386
6,509,25,15296943,8849117,4208
7,462,11,15476018,9028192,3960
8,390,21,15655240,9207414,3648
9,398,15,15834445,9386619,3415


In [5]:
CSV.write("symulacja_90proc.csv", y1)

"symulacja_90proc.csv"

Ponizej druga symulacja, w ktorej chec zaszczepienie deklaruje 10% jeszcze nie zaszczepionych.

In [6]:
x2 = zaraza(szczepienia, 365, 0.10)

test2
test3
test4
test5
test6
test7
test8
test9
test10
test11
test12
test13
test14
test15
test16
test17
test18
test19
test20
test21
test22
test23
test24
test25
test26
test27
test28
test29
test30
test31
test32
test33
test34
test35
test36
test37
test38
test39
test40
test41
test42
test43
test44
test45
test46
test47
test48
test49
test50
test51
test52
test53
test54
test55
test56
test57
test58
test59
test60
test61
test62
test63
test64
test65
test66
test67
test68
test69
test70
test71
test72
test73
test74
test75
test76
test77
test78
test79
test80
test81
test82
test83
test84
test85
test86
test87
test88
test89
test90
test91
test92
test93
test94
test95
test96
test97
test98
test99
test100
test101
test102
test103
test104
test105
test106
test107
test108
test109
test110
test111
test112
test113
test114
test115
test116
test117
test118
test119
test120
test121
test122
test123
test124
test125
test126
test127
test128
test129
test130
test131
test132
test133
test134
test135
test136
test137
test138
test139
te

366×6 Matrix{Int64}:
 565  91  14224874   7777048  5642  1202
 612  46  14581575   8133749  4995  1213
 533  29  14760701   8312875  4789   710
 512  28  14939315   8491489  4540   733
 499  15  15117895   8670069  4315   709
 457  28  15296976   8849150  4068   676
 445  14  15476057   9028231  3814   685
 390  14  15654587   9206761  3502   688
 364  27  15834229   9386403  3246   593
 340  13  16012542   9564716  3050   523
 348  14  16191643   9743817  2882   502
 307  17  16370554   9922728  2684   488
 276  15  16549017  10101191  2505   440
   ⋮                                   ⋮
   0   0  20660330  14212504     0     0
   0   0  20660330  14212504     0     0
   0   0  20660330  14212504     0     0
   0   0  20660330  14212504     0     0
   0   0  20660330  14212504     0     0
   0   0  20660330  14212504     0     0
   0   0  20660330  14212504     0     0
   0   0  20660330  14212504     0     0
   0   0  20660330  14212504     0     0
   0   0  20660330  14212504     0  

In [7]:
y2 = DataFrame(
    dzienne_zarazenia = x2[1:end,1],
    dzienna_smierc = x2[1:end,2],
    lacznie_1_dawka = x2[1:end,3],
    lacznie_2_dawka = x2[1:end,4],
    teraz_zarazeni = x2[1:end,5],
    ozdrowiency = x2[1:end,6]
    )

,dzienne_zarazenia,dzienna_smierc,lacznie_1_dawka,lacznie_2_dawka,teraz_zarazeni
,Int64,Int64,Int64,Int64,Int64
1,565,91,14224874,7777048,5642
2,612,46,14581575,8133749,4995
3,533,29,14760701,8312875,4789
4,512,28,14939315,8491489,4540
5,499,15,15117895,8670069,4315
6,457,28,15296976,8849150,4068
7,445,14,15476057,9028231,3814
8,390,14,15654587,9206761,3502
9,364,27,15834229,9386403,3246


In [8]:
CSV.write("symulacja_10proc.csv", y2)

"symulacja_10proc.csv"

Ponizej trzecia symulacja, w ktorej chec zaszczepienie deklaruje az 40% jeszcze nie zaszczepionych.

In [9]:
x3 = zaraza(szczepienia, 365, 0.40)

test2
test3
test4
test5
test6
test7
test8
test9
test10
test11
test12
test13
test14
test15
test16
test17
test18
test19
test20
test21
test22
test23
test24
test25
test26
test27
test28
test29
test30
test31
test32
test33
test34
test35
test36
test37
test38
test39
test40
test41
test42
test43
test44
test45
test46
test47
test48
test49
test50
test51
test52
test53
test54
test55
test56
test57
test58
test59
test60
test61
test62
test63
test64
test65
test66
test67
test68
test69
test70
test71
test72
test73
test74
test75
test76
test77
test78
test79
test80
test81
test82
test83
test84
test85
test86
test87
test88
test89
test90
test91
test92
test93
test94
test95
test96
test97
test98
test99
test100
test101
test102
test103
test104
test105
test106
test107
test108
test109
test110
test111
test112
test113
test114
test115
test116
test117
test118
test119
test120
test121
test122
test123
test124
test125
test126
test127
test128
test129
test130
test131
test132
test133
test134
test135
test136
test137
test138
test139
te

366×6 Matrix{Int64}:
 565  91  14224874   7777048  5642  1202
 622  33  14582010   8134184  4953  1278
 538  26  14761086   8313260  4735   730
 505  22  14939474   8491648  4485   733
 452  24  15118531   8670705  4242   671
 461  21  15297208   8849382  3997   685
 431  16  15475747   9027921  3721   691
 371  22  15654928   9207102  3420   650
 352  19  15833643   9385817  3150   603
 321  26  16012627   9564801  2929   516
 335  12  16191533   9743707  2769   483
 324  13  16370059   9922233  2631   449
 272  13  16548901  10101075  2436   454
   ⋮                                   ⋮
   0   0  20660196  14212370     0     0
   0   0  20660196  14212370     0     0
   0   0  20660196  14212370     0     0
   0   0  20660196  14212370     0     0
   0   0  20660196  14212370     0     0
   0   0  20660196  14212370     0     0
   0   0  20660196  14212370     0     0
   0   0  20660196  14212370     0     0
   0   0  20660196  14212370     0     0
   0   0  20660196  14212370     0  

In [10]:
y3 = DataFrame(
    dzienne_zarazenia = x3[1:end,1],
    dzienna_smierc = x3[1:end,2],
    lacznie_1_dawka = x3[1:end,3],
    lacznie_2_dawka = x3[1:end,4],
    teraz_zarazeni = x3[1:end,5],
    ozdrowiency = x3[1:end,6]
    )

,dzienne_zarazenia,dzienna_smierc,lacznie_1_dawka,lacznie_2_dawka,teraz_zarazeni
,Int64,Int64,Int64,Int64,Int64
1,565,91,14224874,7777048,5642
2,622,33,14582010,8134184,4953
3,538,26,14761086,8313260,4735
4,505,22,14939474,8491648,4485
5,452,24,15118531,8670705,4242
6,461,21,15297208,8849382,3997
7,431,16,15475747,9027921,3721
8,371,22,15654928,9207102,3420
9,352,19,15833643,9385817,3150


In [11]:
CSV.write("symulacja_40proc.csv", y3)

"symulacja_40proc.csv"